# Carbon Aware SDK Demo Notebook

This notebook is supposed to give you an introduction to using the Carbon Aware SDK, by interacting with it deployed as a Web API. For a guide on setting it up as a Web API, see the GettingStarted.md guide in the main repo of the project: https://github.com/Green-Software-Foundation/carbon-aware-sdk/blob/dev/GettingStarted.md

In [ ]:
# Get all regions to find worst possible region:

In [1]:
# Getting data with the Carbon Aware SDK
import time
import openapi_client
from pprint import pprint
from openapi_client.api import carbon_aware_api
from openapi_client.model.emissions_data import EmissionsData
from  dateutil.parser import parse
# Defining the host is optional and defaults to http://localhost
# See configuration.py for a list of all supported configuration parameters.
configuration = openapi_client.Configuration(
        host = "https://carbon-aware-sdk.azurewebsites.net"
        #host = "http://localhost:5073"
)


 
# Enter a context with an instance of the API client
with openapi_client.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    api_instance = carbon_aware_api.CarbonAwareApi(api_client)
    location = "westus" # str |  (optional)
    time = parse('2022-07-22T10:30:00.00Z') # datetime |  (optional)
    to_time = parse('2022-07-22T11:00:00.00Z') # datetime |  (optional)
    duration_minutes = 0 # int |  (optional) (default to 0)

    try:
        api_response = api_instance.get_emissions_data_for_location_by_time(location=location, time=time, to_time=to_time, duration_minutes=duration_minutes)

        pprint(api_response)
    except openapi_client.ApiException as e:
        print("Exception when calling CarbonAwareApi->emissions_bylocation_get: %s\n" % e)



[{'duration': '00:05:00',
 'location': 'CAISO_NORTH',
 'rating': 409.59391011,
 'time': datetime.datetime(2022, 7, 22, 11, 0, tzinfo=tzutc())},
 {'duration': '00:05:00',
 'location': 'CAISO_NORTH',
 'rating': 406.87235589,
 'time': datetime.datetime(2022, 7, 22, 10, 55, tzinfo=tzutc())},
 {'duration': '00:05:00',
 'location': 'CAISO_NORTH',
 'rating': 406.87235589,
 'time': datetime.datetime(2022, 7, 22, 10, 50, tzinfo=tzutc())},
 {'duration': '00:05:00',
 'location': 'CAISO_NORTH',
 'rating': 406.87235589,
 'time': datetime.datetime(2022, 7, 22, 10, 45, tzinfo=tzutc())},
 {'duration': '00:05:00',
 'location': 'CAISO_NORTH',
 'rating': 406.87235589,
 'time': datetime.datetime(2022, 7, 22, 10, 40, tzinfo=tzutc())},
 {'duration': '00:05:00',
 'location': 'CAISO_NORTH',
 'rating': 406.87235589,
 'time': datetime.datetime(2022, 7, 22, 10, 35, tzinfo=tzutc())},
 {'duration': '00:05:00',
 'location': 'CAISO_NORTH',
 'rating': 407.32594826,
 'time': datetime.datetime(2022, 7, 22, 10, 30, tzin

# By Making Applications Carbon Aware, we can decrease carbon footprint of software by up to 76%!
Based on example of moving from westcentralus to francecentral - times used for calculation:
- westcentralus: 2022-08-15 20:55
- francecentral: 2022-08-15 13:45

## Scenario:
A software engineer is running data processing job at 8:55 pm every day. The job is run in `westcentralus` Azure Data centre. He suspects that running the task at a different time of day, might produce less carbon emissions, since there is more solar energy in the grid during the day.
1. He wants to check, what is the current carbon intensity in that location.
2. He also would like to know, if it is better for the planet 🌍, to run this task at a different time of day.

He decides to use the Carbon Aware SDK to satisfy his curiosity, and possibly build a business case to move the job to another time, if his findings confirm his suspicions. He has cloned the [repository](https://github.com/Green-Software-Foundation/carbon-aware-sdk), and hosted it locally as a Web API on https://localhost:5073


1. He starts by running a query on the `/emissions/bylocation` endpoint of the Carbon Aware SDK, for the region `westcentralus` and time 8:55pm:

In [5]:
## Engineer's preperations:
# Getting data with the Carbon Aware SDK
import os
import pytz
import datetime
import openapi_client
from pprint import pprint
from openapi_client.api import carbon_aware_api
from openapi_client.model.emissions_data import EmissionsData
from  dateutil.parser import parse
from dotenv import load_dotenv

# Load env variables:
load_dotenv()
try:
    host_url = os.environ["SDK_WEB_HOST"]
except KeyError:
    print("SDK_WEB_HOST environment var not set, using localhost")
    host_url = "http://localhost:5073"
    
# Defining the host is optional and defaults to http://localhost
# See configuration.py for a list of all supported configuration parameters.
configuration = openapi_client.Configuration(
        host = host_url
)


Running the actual query, with parameters:
- location: `westcentralus`
- start_time: 2022-08-15 21:00
- end_time: 2022-08-15 20:55
- duration_minutes: 0

In [11]:
# Enter a context with an instance of the API client
with openapi_client.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    api_instance = carbon_aware_api.CarbonAwareApi(api_client)
    location = "westcentralus"               # str |  (required)
    start_time = parse('2022-08-15T21:00')   # datetime |  (optional)
    to_time = parse('2022-08-15T20:55')      # datetime |  (optional)
    duration_minutes = 0                     # int |  (optional) (default to 0)

    try:
        api_response = api_instance.get_emissions_data_for_location_by_time(location=location, time=start_time, to_time=to_time, duration_minutes=duration_minutes)

        pprint(api_response)
    except openapi_client.ApiException as e:
        print("Exception when calling CarbonAwareApi->emissions_bylocation_get: %s\n" % e)

[{'duration': '00:05:00',
 'location': 'PACE',
 'rating': 928.50358139,
 'time': datetime.datetime(2022, 8, 15, 20, 55, tzinfo=tzutc())}]


From the query above, the software engineer can see that the carbon intensity in `westcentralus` at 8:55pm is `928.5` grams CO2 per kWh. That's a lot!

He wonders whether running it at a different time of day might improve that value, so he makes a query to find the best time of day for running his job, by polling the `/emissions/bylocations/best` endpoint. (It will return a single location and time with the lowest carbon intensity in a given time window). He runs it, telling the SDK to look for best time between 12am on 15.08.2022 and 11:59pm on 15.08.2022.

parameters:
- locations: [`westcentralus`]
- start_time: 2022-08-15 00:00
- end_time: 2022-08-15 23:59
- duration_minutes: 0

In [37]:
# Enter a context with an instance of the API client
with openapi_client.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    api_instance = carbon_aware_api.CarbonAwareApi(api_client)
    locations = ["westcentralus"    ]        # List[str] |  (required)
    start_time = parse('2022-08-15T00:00')   # datetime |  (optional)
    to_time = parse('2022-08-15T20:55')      # datetime |  (optional)
    duration_minutes = 0                     # int |  (optional) (default to 0)

    try:
        api_response = api_instance.get_best_emissions_data_for_locations_by_time(location=locations, time=start_time, to_time=to_time, duration_minutes=duration_minutes)

        print("Best time for running the job")
        pprint(api_response)
    except openapi_client.ApiException as e:
        print("Exception when calling CarbonAwareApi->emissions_bylocation_get: %s\n" % e)

Best time for running the job
{'duration': '00:05:00',
 'location': 'PACE',
 'rating': 493.96209093,
 'time': datetime.datetime(2022, 8, 14, 23, 55, tzinfo=tzutc())}


## Outcome of time shifting: Carbon footprint decrased by up to 47%!
Engineer's short experiment showed him that moving the job to a different time (In this case, 11:55 pm UTC), he can decrease the carbon emissions from `928.5` gram CO2 per kWh to `494.0`!

Below, we can see data for this region for past 30 days with the original and time shifted values of carbon emissions in `westcentralus`.

In [47]:
# Enter a context with an instance of the API client
with openapi_client.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    api_instance = carbon_aware_api.CarbonAwareApi(api_client)
    location = "westcentralus"               # str |  (required)
    start_time = parse('2022-07-17T00:05')   # datetime |  (optional)
    to_time = parse('2022-08-15T23:59')      # datetime |  (optional)
    duration_minutes = 0                     # int |  (optional) (default to 0)

    try:
        api_response = api_instance.get_emissions_data_for_location_by_time(location=location, time=start_time, to_time=to_time, duration_minutes=duration_minutes)
        data_westcentralus = api_response[:]
        pprint(api_response[:5])
    except openapi_client.ApiException as e:
        print("Exception when calling CarbonAwareApi->emissions_bylocation_get: %s\n" % e)

[{'duration': '00:05:00',
 'location': 'PACE',
 'rating': 493.96209093,
 'time': datetime.datetime(2022, 8, 15, 23, 55, tzinfo=tzutc())},
 {'duration': '00:05:00',
 'location': 'PACE',
 'rating': 418.21216514,
 'time': datetime.datetime(2022, 8, 15, 23, 50, tzinfo=tzutc())},
 {'duration': '00:05:00',
 'location': 'PACE',
 'rating': 619.6071774200001,
 'time': datetime.datetime(2022, 8, 15, 23, 45, tzinfo=tzutc())},
 {'duration': '00:05:00',
 'location': 'PACE',
 'rating': 584.22697256,
 'time': datetime.datetime(2022, 8, 15, 23, 40, tzinfo=tzutc())},
 {'duration': '00:05:00',
 'location': 'PACE',
 'rating': 608.72096054,
 'time': datetime.datetime(2022, 8, 15, 23, 35, tzinfo=tzutc())}]


Grouping the data:

In [51]:
data_8_55_westcentralus = list(filter(lambda x: x['time'].strftime('%H:%M:%S') == parse("2022-08-15T20:55").strftime('%H:%M:%S'), data_westcentralus))
print("8:55 in west_centralus length: ", len(data_8_55_westcentralus))
pprint(data_8_55_westcentralus[:3])
data_11_55_westcentralus = list(filter(lambda x: x['time'].strftime('%H:%M:%S') == parse("2022-08-15T23:55").strftime('%H:%M:%S'), data_westcentralus))
print("8:55 in west_centralus length: ", len(data_11_55_westcentralus))
pprint(data_11_55_westcentralus[:3])


8:55 in west_centralus length:  30
[{'duration': '00:05:00',
 'location': 'PACE',
 'rating': 928.50358139,
 'time': datetime.datetime(2022, 8, 15, 20, 55, tzinfo=tzutc())},
 {'duration': '00:05:00',
 'location': 'PACE',
 'rating': 928.50358139,
 'time': datetime.datetime(2022, 8, 14, 20, 55, tzinfo=tzutc())},
 {'duration': '00:05:00',
 'location': 'PACE',
 'rating': 824.63092866,
 'time': datetime.datetime(2022, 8, 13, 20, 55, tzinfo=tzutc())}]
8:55 in west_centralus length:  30
[{'duration': '00:05:00',
 'location': 'PACE',
 'rating': 493.96209093,
 'time': datetime.datetime(2022, 8, 15, 23, 55, tzinfo=tzutc())},
 {'duration': '00:05:00',
 'location': 'PACE',
 'rating': 493.96209093,
 'time': datetime.datetime(2022, 8, 14, 23, 55, tzinfo=tzutc())},
 {'duration': '00:05:00',
 'location': 'PACE',
 'rating': 493.96209093,
 'time': datetime.datetime(2022, 8, 13, 23, 55, tzinfo=tzutc())}]


Data to be plotted, see: https://www.machinelearningplus.com/plots/top-50-matplotlib-visualizations-the-master-plots-python/#35.-Time-Series-Plot